<a href="https://colab.research.google.com/github/SHodapp117/Data-Visualization/blob/main/WK8_SHHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import csv
import pandas as pd
# Load JSON data
with open("duto_guerra_tweets.json", "r") as json_file:
    data = json.load(json_file)

# Create DataFrame
tweets_df = pd.json_normalize(data)

tweets_df["entities.hashtags"].head(n=20)

0                                                   []
1                                                   []
2                                                   []
3                                                   []
4       [{'text': 'dataviz', 'indices': ['40', '48']}]
5                                                   []
6                                                   []
7                                                   []
8                                                   []
9                                                   []
10                                                  []
11                                                  []
12                                                  []
13                                                  []
14                                                  []
15                                                  []
16                                                  []
17    [{'text': 'dataviz', 'indices': ['225', '233']}]
18    [{'t

In [4]:
import networkx as nx
import pandas as pd
import altair as alt

# Extract hashtags
def extract_hashtags(text):
    words = text.split()
    hashtags = [word[1:] for word in words if word.startswith('#')]
    return hashtags

# Preprocess the data
tweets_df = tweets_df[tweets_df['entities.hashtags'].apply(lambda x: len(x) > 0)]

G = nx.Graph()

# Iterate through the hashtags in each tweet and build the graph
for hashtag_list in tweets_df['entities.hashtags']:
    hashtag_texts = [hashtag_dict['text'] for hashtag_dict in hashtag_list]
    for a in hashtag_texts:
        G.add_node(a)
        for b in hashtag_texts:
            if a != b:
                G.add_edge(a, b)

# Visualization of the network
g = nx.spring_layout(G, seed=42)

# Nodes
nodes = pd.DataFrame(g.keys(), columns=['node'])
nodes['x'] = nodes['node'].apply(lambda x: g[x][0])
nodes['y'] = nodes['node'].apply(lambda x: g[x][1])

# Edges
edges = pd.DataFrame(G.edges(), columns=['source', 'target'])

# Chart for nodes
nodes_chart = alt.Chart(nodes).mark_circle(size=300, color='skyblue').encode(
    x='x:Q',
    y='y:Q',
    tooltip='node:N'
).interactive()

# Chart for edges
edges_chart = alt.Chart(edges).mark_rule().encode(
    x=alt.X('source:N', title='Hashtag 1'),
    x2='target:N',
    y=alt.Y('source:N', title='Hashtag 2'),
    y2='target:N'
)

network_chart = edges_chart + nodes_chart

network_chart.properties(
    width=800,
    height=800,
    title='Hashtag Co-occurrence Network'
)


alt.LayerChart(...)